<a href="https://colab.research.google.com/github/JoaoMouraSS/Desafios/blob/main/main2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/Competição QConcurso/Dataset_model.csv')

In [ ]:
v_de_interesse = df1[['region','gp:college type','gp:degree course','gp:previous experience','gp:school type','commented_by_professor','difficulty',
'examining_board_id','institute_id','knowledge_area_id','modality_id','nullified','outdated','scholarity_id']]

meta = df1['acertou']

In [ ]:
for i in v_de_interesse:
    if v_de_interesse[i].dtypes == 'object':
        v_de_interesse[i].fillna('nenhum', inplace=True)        
    elif v_de_interesse[i].dtypes == 'float64':
        v_de_interesse[i].fillna(0,inplace = True)
        v_de_interesse[i].astype(int)
    else:
        pass

In [ ]:
# tranformando variáveis float64 em int64
for i in v_de_interesse:
    if v_de_interesse[i].dtypes == 'float64':
        v_de_interesse[i] = np.array(v_de_interesse[i]).astype(int)

In [ ]:
# transformando variável categorica em int
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for i in v_de_interesse:
    if v_de_interesse[i].dtypes == 'object':
        v_de_interesse[i] = encoder.fit_transform(v_de_interesse[i])
    else:
        pass

# normalizando as variáveis
from sklearn.preprocessing import Normalizer
norma = Normalizer()
values = Normalizer().fit_transform(v_de_interesse.values)

In [ ]:
# Separar os dados para fazer o primeiro treino
from sklearn.model_selection import train_test_split
treino_x,teste_x,treino_y,teste_y = train_test_split(values,meta,random_state=1,test_size=0.3,stratify=meta)

In [ ]:
# Construindo a base line
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score

base_line = DummyClassifier().fit(treino_x,treino_y)
pred_baseline = base_line.predict(teste_x)

f1_score(y_pred=pred_baseline, y_true=teste_y)

In [ ]:
# Criando um modelo
import xgboost as xgb
model_xgb = xgb.XGBClassifier()
modelo_xgb = model_xgb.fit(treino_x,treino_y)
pred_xgb = modelo_xgb.predict(teste_x)

f1_score(y_pred=pred_xgb, y_true=teste_y)

In [21]:
df_submit = pd.read_excel('/content/Submit.xlsx')

In [ ]:
v_de_interesse = df_submit[['region','gp:college type','gp:degree course','gp:previous experience','gp:school type','commented_by_professor','difficulty',
'examining_board_id','institute_id','knowledge_area_id','modality_id','nullified','outdated','scholarity_id']]

for i in v_de_interesse:
    if v_de_interesse[i].dtypes == 'object':
        v_de_interesse[i].fillna('nenhum', inplace=True)        
    elif v_de_interesse[i].dtypes == 'float64':
        v_de_interesse[i].fillna(0,inplace = True)
        v_de_interesse[i].astype(int)
    else:
        pass

# tranformando variáveis float64 em int64
for i in v_de_interesse:
    if v_de_interesse[i].dtypes == 'float64':
        v_de_interesse[i] = np.array(v_de_interesse[i]).astype(int)

# transformando variável categorica em int
encoder = LabelEncoder()
for i in v_de_interesse:
    if v_de_interesse[i].dtypes == 'object':
        v_de_interesse[i] = encoder.fit_transform(v_de_interesse[i])
    else:
        pass

# normalizando as variáveis
norma = Normalizer()
values = Normalizer().fit_transform(v_de_interesse.values)

In [24]:
submit = modelo_xgb.predict(values)

In [32]:
df_result = pd.DataFrame()

df_result['novo_user_id'] = df_submit['novo_user_id'].astype(int)
df_result['resultado'] = submit

In [33]:
df_result.head()

,novo_user_id,resultado
0,1,1
1,2,1
2,3,0
3,4,1
4,5,0


In [34]:
df_result.to_csv('submit.csv',header=True)